# Steal The Files - Documentation (EN)
Client | Server | Core - JavaFX 2D platformer

**Contents**
1. Overview & Architecture
2. Setup & Start
3. Game Logic: Steps, Decisions, Notes + Code
4. UI/Navigation: Steps, Decisions, Notes + Code
5. Level Build: Steps, Decisions, Notes + Code
6. Audio & Settings + Code
7. Networking & Data + Code
8. Debugging & Open Points + Code
9. Roadmap Nuggets

## 1. Overview & Architecture
- Goal: USB -> unlock files -> collect folders -> survive hazards -> touch finish.
- Level selection:
```java
switch (config.getString("currentLevel")) {
    case "Tutorial" -> setCurrentLevel(new TutorialLevel());
    case "Second"   -> setCurrentLevel(new SecondLevel());
    case "Boss"     -> setCurrentLevel(new BossLevel());
}
```
- Game loop thread + UI on JavaFX:
```java
while (running) {
    double delta = (now - lastTime) / 1_000_000_000.0;
    lastTime = now;
    Platform.runLater(() -> currentScreen.update(delta));
    Thread.sleep(5);
}
```

## 2. Setup & Start
- Build: 
```bash
mvn clean install
```
- Run (IDE): Server `de.cyzetlc.hsbi.Server`, Client `de.cyzetlc.hsbi.game.Game`.
- Config excerpt:
```json
{
  "currentLevel": "Tutorial",
  "soundVolume": 0.6,
  "soundMuted": false,
  "messages": { "game.prefix": "HSBI-GAME" },
  "mysql": { "hostname": "db-host", "database": "hsbi", "username": "user", "password": "<secret>", "port": 3306, "poolSize": 3 },
  "tutorialFinished": false
}
```

## 3. Game Logic: Steps, Decisions, Notes + Code
**Flow (GameScreen)**
- Input/toggles:
```java
boolean f2 = screenManager.getInputManager().pollJustPressed(KeyCode.F2);
if (f2) showDebugBar = !showDebugBar;
```
- Movement/physics:
```java
dy += Game.gravity * delta;
double nextX = x + dx;
double nextY = y + dy;
Rectangle2D nextBounds = new Rectangle2D(nextX, nextY, player.getWidth(), player.getHeight());
```
- NoClip/GodMode (F3):
```java
if (f3) { player.setNoClip(!player.isNoClipEnabled()); player.setGodMode(player.isNoClipEnabled()); }
if (player.isNoClipEnabled()) { /* free fly */ return; }
```
- RobotEnemy firing:
```java
boolean close = Math.abs(player.getLocation().getX() - getLocation().getX()) < 440;
if (close && fireTimer >= fireCooldown) {
    fireTimer = 0;
    return new LaserBlock(new Location(spawnX, eyeY), dir, 320);
}
```
- GasBarrier interaction:
```java
if (block instanceof GasBarrierBlock barrier && interactPressed && player.hasFlipper()) {
    barrier.deactivate();
    continue;
}
```
- Perk reset:
```java
PauseTransition delay = new PauseTransition(Duration.seconds(10));
delay.setOnFinished(e -> Game.jumpPower = 800);
delay.play();
```
**Design notes**: USB gates folders; fixed 4s laser lifetime; delta physics + separate UI thread.
**Tech issues**: pause overlay only; `Level.update()` unused; no camera bounds; LevelFinished stats hardcoded.

## 4. UI/Navigation: Steps, Decisions, Notes + Code
**Build**
- LoadingScreen:
```java
Timeline tl = new Timeline(new KeyFrame(Duration.millis(3000), kvWidth));
tl.setOnFinished(e -> screenManager.showScreen(mainMenu));
```
- SettingsScreen volume/mute:
```java
volumeSlider.valueProperty().addListener((o, ov, nv) -> {
    SoundManager.setVolume(nv.doubleValue());
    updateVolumeLabel();
});
muteBtn.setOnAction(e -> {
    SoundManager.setMuted(!SoundManager.isMuted());
    volumeSlider.setDisable(SoundManager.isMuted());
});
```
- HUD update (files/health):
```java
this.totalFolderCount = countFolderBlocks();
filesProgressLbl.setText("Files: " + countActiveFolders() + "/" + totalFolderCount);
updateHealth();
```
- Pause overlay:
```java
private void togglePause() {
    paused = !paused;
    pauseOverlay.setVisible(paused);
}
```
**Design notes**: fullscreen, no OS chrome; HUD left, health right; pause kept visual for debugging.
**Tech issues**: LevelFinished placeholders; no resize handling.

## 5. Level Build: Steps, Decisions, Notes + Code
**TutorialLevel**
```java
platforms.add(new Platform(0, h-300, 450, 600, root));
blocks.add(new USBStickBlock(new Location(200, h-360)));
blocks.add(new FloatingPlatformBlock(new Location(940, h-340), new Location(1224, h-340), 120));
placeLavaBetweenPlatforms(h);
```
**SecondLevel**
```java
blocks.add(new FlipperItem(new Location(220, h-320)));
blocks.add(new RobotEnemyBlock(new Location(5000, h-396), 500, 180));
// blocks.add(new GasBarrierBlock(new Location(4550, h-428), 64, 128));
blocks.add(new FolderBlock(new Location(3600, h-530)));
```
**BossLevel**
```java
blocks.add(new RobotEnemyBlock(new Location(250, h-396), 500, 180));
blocks.add(new RobotEnemyBlock(new Location(1400, h-396), 500, 180));
blocks.add(new FinishBlock(new Location(1800, h-390)));
```
**Design notes**: USB before folders; early flipper in SecondLevel; mix of floating platform + enemy.
**Tech issues**: hardcoded coords; no camera bounds; Level.update unused; GasBarrier commented.

## 6. Audio & Settings + Code
- Persist volume/mute:
```java
public static void setVolume(double volume) {
    globalVolume = Math.max(0, Math.min(1, volume));
    Game.getInstance().getConfig().getObject().put("soundVolume", volume);
    Game.getInstance().getConfig().save();
}
public static void setMuted(boolean muted) {
    SoundManager.muted = muted;
    Game.getInstance().getConfig().getObject().put("soundMuted", muted);
    Game.getInstance().getConfig().save();
}
```
- Ducking:
```java
SoundManager.playWithDuck(Sound.JUMP_BOOST, 1.0, 0.06);
```
- Music loop:
```java
SoundManager.playBackground(Music.MENU, true);
```

## 7. Networking & Data + Code
- Client connect:
```java
socket = new Socket(InetAddress.getByName("localhost"), 25570);
sendPacket(new ClientLoginPacket(thePlayer.getUuid()));
networkExecutor.execute(new ReceiverTask());
networkExecutor.execute(new SenderTask());
```
- Heartbeat sender:
```java
while (!socket.isClosed()) {
    sendPacket(new ClientDataPacket(thePlayer.getUuid()));
    Thread.sleep(50);
}
```
- Server handler:
```java
int bytesRead = dis.read(received);
Packet packet = SerializationUtils.deserialize(actualReceivedData, Packet.class);
if (((EventCancelable) new ReceivePacketEvent(packet, socket).call()).isCancelled()) break;
```
- Score sending:
```java
sendPacket(new ClientSubmitScorePacket(thePlayer.getUuid(), score, "Level_1"));
```

## 8. Debugging & Open Points + Code
- Toggles:
```java
if (f1) showTooltips = !showTooltips;
if (f2) showDebugBar = !showDebugBar;
if (f3) { player.setNoClip(!player.isNoClipEnabled()); player.setGodMode(player.isNoClipEnabled()); }
```
- Game over check:
```java
if (player.getHealth() <= 0) { handleGameOver(); return; }
```
- Open: pause overlay only; `Level.update()` empty; camera clamps missing; GasBarrier commented; LevelFinished stats placeholder.

## 9. Roadmap Nuggets
- Short term: real pause, real LevelFinished stats, camera bounds, enable GasBarrier path.
- Mid term: use Level.update for movers/timers; externalize levels (JSON/editor); HUD timer/score.
- Long term: complete multiplayer/highscore DB flow; backend-driven achievements/server list.